<div align="center" id="top">
 <img src="https://socialify.git.ci/julep-ai/julep/image?description=1&descriptionEditable=Rapidly%20build%20AI%20workflows%20and%20agents&font=Source%20Code%20Pro&logo=https%3A%2F%2Fraw.githubusercontent.com%2Fjulep-ai%2Fjulep%2Fdev%2F.github%2Fjulep-logo.svg&owner=1&forks=1&pattern=Solid&stargazers=1&theme=Auto" alt="julep" width="640" height="320" />
</div>

<p align="center">
  <br />
  <a href="https://docs.julep.ai" rel="dofollow">Explore Docs (wip)</a>
  ·
  <a href="https://discord.com/invite/JTSBGRZrzj" rel="dofollow">Discord</a>
  ·
  <a href="https://x.com/julep_ai" rel="dofollow">𝕏</a>
  ·
  <a href="https://www.linkedin.com/company/julep-ai" rel="dofollow">LinkedIn</a>
</p>

<p align="center">
    <a href="https://www.npmjs.com/package/@julep/sdk"><img src="https://img.shields.io/npm/v/%40julep%2Fsdk?style=social&amp;logo=npm&amp;link=https%3A%2F%2Fwww.npmjs.com%2Fpackage%2F%40julep%2Fsdk" alt="NPM Version"></a>
    <span>&nbsp;</span>
    <a href="https://pypi.org/project/julep"><img src="https://img.shields.io/pypi/v/julep?style=social&amp;logo=python&amp;label=PyPI&amp;link=https%3A%2F%2Fpypi.org%2Fproject%2Fjulep" alt="PyPI - Version"></a>
    <span>&nbsp;</span>
    <a href="https://hub.docker.com/u/julepai"><img src="https://img.shields.io/docker/v/julepai/agents-api?sort=semver&amp;style=social&amp;logo=docker&amp;link=https%3A%2F%2Fhub.docker.com%2Fu%2Fjulepai" alt="Docker Image Version"></a>
    <span>&nbsp;</span>
    <a href="https://choosealicense.com/licenses/apache/"><img src="https://img.shields.io/github/license/julep-ai/julep" alt="GitHub License"></a>
</p>

## Task Definition: Crawling and RAG

### Overview

This task implements an automated system to index and process certain website. The system crawls a website, extracts relevant information, and creates a searchable knowledge base that can be queried programmatically.

### Task Tools:

- **spider_crawler**: Web crawler component for systematically traversing the website
- **create_agent_doc**: Document processor for converting web content into indexed, searchable documents

### Task Input:

Required parameter:
- **url**: Entry point URL for the crawler (e.g., "https://julep.ai/")

### Task Output:

- Indexed knowledge base containing processed website
- Query interface for programmatic access to the crawled information


## Implementation

To recreate the notebook and see the code implementation for this task, you can access the Google Colab notebook using the link below:

<a target="_blank" href="https://colab.research.google.com/github/julep-ai/julep/blob/dev/cookbooks/10-crawling-and-rag.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### Additional Information

For more details about the task or if you have any questions, please don't hesitate to contact the author:

**Author:** Julep AI  
**Contact:** [hey@julep.ai](mailto:hey@julep.ai) or  <a href="https://discord.com/invite/JTSBGRZrzj" rel="dofollow">Discord</a>

Installing the Julep Client

In [ ]:
# Install the Julep SDK
!pip install --upgrade julep --quiet

#### NOTE:

- UUIDs are generated for both the agent and task to uniquely identify them within the system.
- Once created, these UUIDs should remain unchanged for simplicity.
- Altering a UUID will result in the system treating it as a new agent or task.
- If a UUID is changed, the original agent or task will continue to exist in the system alongside the new one.

In [1]:
# Global UUID is generated for agent and task
from dotenv import load_dotenv
from julep import Client
import os

load_dotenv(override=True)

# Set your API keys
JULEP_API_KEY = os.getenv("JULEP_API_KEY")

# Create a Julep client
client = Client(api_key=JULEP_API_KEY, environment="production")

## Creating an "agent"

Agent is the object to which LLM settings, like model, temperature along with tools are scoped to.

To learn more about the agent, please refer to the [documentation](https://github.com/julep-ai/julep/blob/dev/docs/julep-concepts.md#agent).

In [2]:
# Create agent
agent = client.agents.create(
    name="Website Crawler",
    about="An AI assistant that can crawl any website and create a knowledge base.",
    model="gpt-4o",
)

In [3]:
num_docs = len(client.agents.docs.list(agent_id=agent.id, limit=100).items)
print(f"Number of documents in the agent's document store: {num_docs}")

Number of documents in the agent's document store: 0


## Defining a Task

Tasks in Julep are Github-Actions-style workflows that define long-running, multi-step actions.

You can use them to conduct complex actions by defining them step-by-step.

To learn more about tasks, please refer to the `Tasks` section in [Julep Concepts](https://github.com/julep-ai/julep/blob/dev/docs/julep-concepts.md#tasks).

In [4]:
import yaml

task_def = yaml.safe_load(f"""
name: Crawl a website and create a agent document

# Define the tools that the agent will use in this workflow
tools:
- name: spider_crawler
  type: integration
  integration:
    provider: spider
    method: crawl
    setup:
      spider_api_key: "{os.getenv('SPIDER_API_KEY')}"

- name : create_agent_doc
  description: Create an agent doc
  type: system
  system:
    resource: agent
    subresource: doc
    operation: create

index_page:

- evaluate:
    documents: _['content']

- over: "[(_0.content, chunk) for chunk in _['documents']]"
  parallelism: 3
  map:
    prompt: 
    - role: user
      content: >-
        <document>
        {{{{_[0]}}}}
        </document>

        Here is the chunk we want to situate within the whole document
        <chunk>
        {{{{_[1]}}}}
        </chunk>

        Please give a short succinct context to situate this chunk within the overall document for the purposes of improving search retrieval of the chunk. 
        Answer only with the succinct context and nothing else. 
    
    unwrap: true
    settings:
      max_tokens: 16000

- evaluate:
    final_chunks: |
      [
        NEWLINE.join([chunk, succint]) for chunk, succint in zip(_1.documents, _)
      ]

# Create a new document and add it to the agent docs store
- over: _['final_chunks']
  parallelism: 3
  map:
    tool: create_agent_doc
    arguments:
      agent_id: "'{agent.id}'"
      data:
        metadata:
          source: "'spider_crawler'"

        title: "'Website Document'"
        content: _

# Define the steps of the workflow
main:

# Define a tool call step that calls the spider_crawler tool with the url input
- tool: spider_crawler
  arguments:
    url: "_['url']" # You can also use 'inputs[0]['url']'
    params:
      request: "'smart_mode'"
      limit: _['pages_limit'] # <--- This is the number of pages to crawl (taken from the input of the task)
      return_format: "'markdown'"
      proxy_enabled: "True"
      filter_output_images: "True" # <--- This is to execlude images from the output
      filter_output_svg: "True" # <--- This is to execlude svg from the output
      # filter_output_main_only: "True"
      readability: "True" # <--- This is to make the output more readable
      sitemap: "True" # <--- This is to crawl the sitemap
      chunking_alg: # <--- Using spider's bysentence algorithm to chunk the output
        type: "'bysentence'"
        value: "15" # <--- This is the number of sentences per chunk

# Evaluate step to document chunks
- foreach:
    in: _['result']
    do:
      workflow: index_page
      arguments:
        content: _.content
""")


<span style="color:olive;">Notes:</span>
- The reason for using the quadruple curly braces `{{{{}}}}` for the jinja template is to avoid conflicts with the curly braces when using the `f` formatted strings in python. [More information here](https://stackoverflow.com/questions/64493332/jinja-templating-in-airflow-along-with-formatted-text)
- The `unwrap: True` in the prompt step is used to unwrap the output of the prompt step (to unwrap the `choices[0].message.content` from the output of the model).


## Creating a task

In [5]:
# Create the task
task = client.tasks.create(
    agent_id=agent.id,
    **task_def
)

## Creating an Execution (Starting from the Homepage)

An execution is a single run of a task. It is a way to run a task with a specific set of inputs.

In [10]:
execution = client.executions.create(
    task_id=task.id,
    input={
        "url": "https://julep.ai/",
        "pages_limit": 5,
    }
)

## Checking execution details and output

There are multiple ways to get the execution details and the output:

1. **Get Execution Details**: This method retrieves the details of the execution, including the output of the last transition that took place.

2. **List Transitions**: This method lists all the task steps that have been executed up to this point in time, so the output of a successful execution will be the output of the last transition (first in the transition list as it is in reverse chronological order), which should have a type of `finish`.


<span style="color:olive;">Note: You need to wait for a few seconds for the execution to complete before you can get the final output, so feel free to run the following cells multiple times until you get the final output.</span>


In [12]:
status = client.executions.get(execution_id=execution.id).status

print("Execution status: ", status)

Execution status:  succeeded


In [13]:
import json
execution_transitions = client.executions.transitions.list(
    execution_id=execution.id, limit=100).items

for index, transition in enumerate(reversed(execution_transitions)):
    print("Index: ", index, "Type: ", transition.type)
    print("output: ", json.dumps(transition.output, indent=2))
    print("-" * 100)

Index:  0 Type:  init
output:  {
  "url": "https://julep.ai/",
  "pages_limit": 5
}
----------------------------------------------------------------------------------------------------
Index:  1 Type:  step
output:  {
  "result": [
    {
      "url": "https://julep.ai/",
      "costs": {
        "ai_cost": 0,
        "file_cost": 0.0004,
        "total_cost": 0.0006,
        "compute_cost": 0.0001,
        "transform_cost": 0.0001,
        "bytes_transferred_cost": 0
      },
      "error": null,
      "status": 200,
      "content": [
        "[\n](./)\n[\nDocs\n](https://docs.julep.ai/)\n[\nGithub\n](https://github.com/julep-ai/julep)\n[\nDashboard\n](https://dashboard.julep.ai/agents/)\nExamples\n[\nDiscord\n](https://discord.gg/2EUJzJU2Yt)\n[\nBook demo\n](< https://calendly.com/ishita-julep>)\n[\nContact Us\n](./contact)\n[\n](./)\n[\n](./)\nAnnouncing our Education+NGO program Eligible organizations get unlimited free credits Please apply[here](https://forms.gle/CCP6sDYeUfAXdUoi7

In [14]:
crawled_pages = [r['url'] for r in execution_transitions[-2].output['result']]

print("Crawled pages: ")
crawled_pages

Crawled pages: 


['https://julep.ai/', 'https://julep.ai/about', 'https://julep.ai/contact']

## Lisitng the Document Store for the Agent

The document store is where the agent stores the documents it has created. Each document has a `title` , `content`, `id`, `metadata`, `created_at` and the `vector embedding` associated with it. This will be used for the retrieval of the documents when the agent is queried.

In [15]:
docs = client.agents.docs.list(agent_id=agent.id, limit=100).items
num_docs = len(docs)
print("Number of documents in the document store: ", num_docs)

Number of documents in the document store:  27


In [72]:
# # # UNCOMMENT THIS TO DELETE ALL THE AGENT'S DOCUMENTS

# for doc in client.agents.docs.list(agent_id=agent.id, limit=1000):
#     client.agents.docs.delete(agent_id=agent.id, doc_id=doc.id)

## Creating a Session

A session is used to interact with the agent. It is used to send messages to the agent and receive responses.
Situation is the initial message that is sent to the agent to set the context for the conversation. Out here you can add more information about the agent and the task it is performing to help the agent answer better. Additionally, you can also define the `search_threshold` and `search_query_chars` which are used to control the retrieval of the documents from the document store which will be used for the retrieval of the documents when the agent is queried.
More information about the session can be found [here](https://github.com/julep-ai/julep/blob/dev/docs/julep-concepts.md#session).

In [19]:
# Custom system template for this particular session to help the agent answer better
system_template = """
You are an AI agent designed to assist users with their queries about a certain website.
Your goal is to provide clear and detailed responses.

**Guidelines**:
1. Assume the user is unfamiliar with the company and products.
2. Thoroughly read and comprehend the user's question.
3. Use the provided context documents to find relevant information.
4. Craft a detailed response based on the context and your understanding of the company and products.
5. Include links to specific website pages for further information when applicable.

**Response format**:
- Use simple, clear language.
- Include relevant website links.

**Important**:
- For questions related to the business, only use the information that are explicitly given in the documents above.
- If the user asks about the business, and it's not given in the documents above, respond with an answer that states that you don't know.
- Use the most recent and relevant data from context documents.
- Be proactive in helping users find solutions.
- Ask for clarification if the query is unclear.
- Inform users if their query is unrelated to the given website.
- Avoid using the following in your response: Based on the provided documents, based on the provided information, based on the documentation... etc.

{%- if docs -%}
**Relevant documents**:{{NEWLINE}}
  {%- for doc in docs -%}
    {{doc.title}}{{NEWLINE}}
    {%- if doc.content is string -%}
      {{doc.content}}{{NEWLINE}}
    {%- else -%}
      {%- for snippet in doc.content -%}
        {{snippet}}{{NEWLINE}}
      {%- endfor -%}
    {%- endif -%}
    {{"---"}}
  {%- endfor -%}

{%- else -%}
There are no documents available for this query.
{%- endif -%}

"""

print(f"Agent created with ID: {agent.id}")

# Create a session for interaction
session = client.sessions.create(
    system_template=system_template,
    agent=agent.id,
    recall_options={
        "mode": "hybrid",
        "num_search_messages": 1,
        "max_query_length": 800,
        "confidence": -0.9,
        "alpha": 0.5,
        "limit": 10,
        "mmr_strength": 0.5,
    },
)

print(f"Session created with ID: {session.id}")

Agent created with ID: 0678f7b6-0865-7f65-8000-e64ed0ca4d19
Session created with ID: 0678f7ce-79fc-7a37-8000-e72d0406c225


## Chatting with the Agent

The chat method is used to send messages to the agent and receive responses. The messages are sent as a list of dictionaries with the `role` and `content` keys.

In [20]:
user_question = "Tell me about Julep"

response = client.sessions.chat(
    session_id=session.id,
    messages=[
        {
            "role": "user",
            "content": user_question,
        }
    ],
    recall=True,
)

print(response.choices[0].message.content)

Julep is a comprehensive platform designed for creating AI agents that possess the ability to remember past interactions and execute complex tasks. It acts as the infrastructure between large language models (LLMs) and your software, providing support for long-term memory and multi-step process management.

Key features of Julep include:

1. **8-Factor Agent Methodology**: This approach incorporates software engineering principles into AI development. It includes managing prompts as code, ensuring model independence, structured reasoning, workflow-based models, and full observability for effective debugging and improvement.

2. **Built-In Reliability**: Julep is equipped with self-healing capabilities such as automatic retries for failed steps, message resending, task recovery, error handling, and real-time monitoring to enhance reliability and manage errors efficiently.

3. **Modular and Flexible Design**: By defining explicit interfaces for tool interactions, Julep ensures that capab

### Check the matched documents

In [21]:
print("Matched docs:\n\n")

for index, doc in enumerate(response.docs):
    print(f"Doc {index + 1}:")
    print(f"Title: {doc.title}")
    print(f"Snippet content:\n{doc.snippet.content}")
    print("-" * 100)

Matched docs:


Doc 1:
Title: Website Document
Snippet content:
It provides the complete infrastructure layer between LLMs and your software, with built-in support for long-term memory and multi-step process management What is Julep Julep is a platform for creating AI agents that remember past interactions and can perform complex tasks It provides the complete infrastructure layer between LLMs and your software, with built-in support for long-term memory and multi-step process management How does Julep handle errors and reliability Julep includes built-in self-healing capabilities:
- Automatic retries for failed steps
- Message resending
- Task recovery
- Error handling
- Real-time monitoring How does Julep handle errors and reliability Julep includes built-in self-healing capabilities:
- Automatic retries for failed steps
- Message resending
- Task recovery
- Error handling
- Real-time monitoring How does Julep handle errors and reliability Julep includes built-in self-healing capabil